In [1]:
# -*- coding: utf-8 -*-
"""과제5 CNN기반 영상분류문제

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1Py2GvZc1GwqNbrR4fuQrHFy6zBEdF1xF
"""

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.optimizers import Adam
import os
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split

(x_train,y_train),(x_test,y_test)=cifar10.load_data()
x_train=x_train.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,10)

x_test=x_test.astype(np.float32)/255.0
y_test=tf.keras.utils.to_categorical(y_test,10)

#x_train, _, y_train,_ = train_test_split(x_train, y_train, test_size=0.5, random_state=42)
x_val, _, y_val,_ = train_test_split(x_test, y_test, test_size=0.8, random_state=42)

input_shape = x_train.shape[1:]

#동일조건 유지해야 하는 변수(두 모델 모두 동일하게 적용해야 함)
g_epoch = 80
g_batch = 128
np.random.seed(42)
tf.random.set_seed(42)

print("reduced train/val size:", len(x_train), len(x_val), "input shape:", input_shape)

# to make this notebook's output stable across runs

tf.__version__

from tensorflow.keras.layers import MaxPooling2D, Dropout, Conv2D

cnn=Sequential()
cnn.add(Conv2D(64,(3,3),activation='relu',input_shape=(32,32,3)))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(128,(3,3),activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(256,(3,3),activation='relu', padding='same'))
cnn.add(Conv2D(256,(3,3),activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Flatten())
cnn.add(Dense(1000,activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(10,activation='softmax'))

cnn.compile(loss='categorical_crossentropy',optimizer=Adam(0.00002),metrics=['accuracy'])
cnn.summary()

hist=cnn.fit(x_train, y_train, batch_size=g_batch, epochs=g_epoch,
             validation_data=(x_val,y_val), verbose=1)

g_org_res=cnn.evaluate(x_test,y_test,verbose=0)
print("Baseline 정확률은",g_org_res[1]*100)

no_class = 10

# for transfer learning only
from tensorflow.keras.applications import ResNet50

# for transfer learning only
transfermodel = ResNet50(weights='imagenet',include_top=False,
                    input_shape=input_shape)
#base_model.trainable=False     # it's up to you

# your model architecture
model=Sequential()
model.add(transfermodel)    # for transfer learning only
model.add(Flatten())        # for transfer learning only
model.add(Dense(1000,activation='relu'))
model.add(Dense(500,activation='relu'))    #layer 하나 더 추가
model.add(Dense(no_class, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer=Adam(0.00002),metrics=['accuracy'])
model.summary()

hist=model.fit(x_train, y_train, batch_size=g_batch, epochs=g_epoch,
             validation_data=(x_val,y_val), verbose=1)

yours=model.evaluate(x_test,y_test,verbose=0)
print("Baseline vs yours: ",g_org_res[1]*100, yours[1]*100)

170508288/170498071 [==============================] - 2s 0us/step
reduced train/val size: 50000 2000 input shape: (32, 32, 3)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 15, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                         